In [1]:
import numpy as np
import pandas as pd
import json
import os

In [13]:
os.getcwd()

'c:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\code'

In [40]:
visitors=pd.read_csv("C:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\source\\visitors.csv")
visitors_questions=pd.read_csv("C:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\source\\visitors_questions.csv")
visitors_answers=pd.read_csv("C:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\source\\visitors_answers.csv")
exhibitors=pd.read_csv("C:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\source\\exhibitors.csv")
exhibitor_categories=pd.read_csv("C:\\Users\\HALID\\Documents\\ITE Assignment\\ITE-data-sciene-assignment\\source\\exhibitor_categories.csv")

In [41]:
visitors['data']=visitors['data'].apply(json.loads)
visitors_exploded=visitors.explode('data').reset_index(drop=True)
visitors_normalized=pd.json_normalize(visitors_exploded['data'])
visitors_final=visitors_exploded.drop(columns='data').join(visitors_normalized)

In [52]:
print("original visitor record count:",visitors['id'].count())
print("original visitor unique count:",visitors['id'].nunique())
print("original visitor column count:",len(visitors.columns))
print("Final visitor record count:",visitors_final['id'].count())
print("Final visitor unique count:",visitors_final['id'].nunique())
print("Final visitor column count:",len(visitors_final.columns))

original visitor record count: 100
original visitor unique count: 100
original visitor column count: 4
Final visitor record count: 2827
Final visitor unique count: 100
Final visitor column count: 8


In [83]:
visitors_final_df=visitors_final.merge(visitors_questions.add_suffix("_questions"),left_on='questionId',right_on="id_questions",how='left').merge(visitors_answers.add_suffix("_answers"),left_on='questionId',right_on="questionId_answers",how='left').drop(['questionId_answers','id_answers','stepId_questions','questionId','id_questions','questionTypeId_questions','stepId_questions'],axis=1)

In [87]:
visitors_final_df[~visitors_final_df['answerTypeId'].isin(['Answer'])]['question_questions'].value_counts()

Series([], Name: question_questions, dtype: int64)

In [89]:
visitors_final_df[~visitors_final_df['answerTypeId'].isin(['Answer'])]['answer_answers'].value_counts()

Series([], Name: answer_answers, dtype: int64)

In [91]:
visitors_final_df['Country']=visitors_final_df.apply(lambda x:x['answerValue'] if x['answerTypeId']=='Country' else None,axis=1)

In [93]:
visitors_final_df['Region']=visitors_final_df.apply(lambda x:x['answerValue'] if x['answerTypeId']=='Region' else None,axis=1)

In [102]:
visitors_final_df[visitors_final_df['id'].isin(['67b70a9f2d21f543a1096602','67b5e25c774d9e718c7541de'])]

,email,gender,id,stepId,answerValue,answerId,answerTypeId,question_questions,answer_answers,Country,Region
0,emilija+100_L8gA@bss.mk,F,67b70a9f2d21f543a1096602,5c8a78336d41a10da4f730ef,,5c8a78336d41a10da4f730f2,Answer,NaN,NaN,Russia,None
1,emilija+100_L8gA@bss.mk,F,67b70a9f2d21f543a1096602,6507da8d9e117216630ef7ed,emilija+100_L8gA@bss.mk,,Answer,NaN,NaN,Russia,None
2,emilija+100_L8gA@bss.mk,F,67b70a9f2d21f543a1096602,6507dfbb9e117216630ef7f3,3138,,Answer,NaN,NaN,Russia,None
3,emilija+100_L8gA@bss.mk,F,67b70a9f2d21f543a1096602,5c8a78336d41a10da4f733ce,Eme,,Answer,NaN,NaN,Russia,None
4,emilija+100_L8gA@bss.mk,F,67b70a9f2d21f543a1096602,5c8a78336d41a10da4f733ce,Jankova,,Answer,NaN,NaN,Russia,None
...,...,...,...,...,...,...,...,...,...,...,...
1277,sergey.usenko_ucwx@ite.group,F,67b5e25c774d9e718c7541de,5c8a78336d41a10da4f730fd,,5c8a78336d41a10da4f73103,Answer,Reason for Attending the Event,To source products and services,New Zealand,None
1278,sergey.usenko_ucwx@ite.group,F,67b5e25c774d9e718c7541de,5c8a78336d41a10da4f730fd,,5c8a78336d41a10da4f73103,Answer,Reason for Attending the Event,To promote products and services,New Zealand,None
1279,sergey.usenko_ucwx@ite.group,F,67b5e25c774d9e718c7541de,5c8a78336d41a10da4f730fd,,5c8a78336d41a10da4f73103,Answer,Reason for Attending the Event,Educational purposes,New Zealand,None
1280,sergey.usenko_ucwx@ite.group,F,67b5e25c774d9e718c7541de,5c8a78336d41a10da4f730fd,,67acaf10197e604dd2722d0b,Answer,NaN,NaN,New Zealand,None


In [ ]:
visitors_final_df['Country']=visitors_final_df.groupby('id')['Country'].transform(lambda x:x.ffill().bfill())
visitors_final_df['Region']=visitors_final_df.groupby('id')['Region'].transform(lambda x:x.ffill().bfill())

In [110]:
visitors_final_df=visitors_final_df[~visitors_final_df['answerTypeId'].isin(['Country','Region'])].reset_index(drop=True)

In [114]:
print(visitors_final_df.groupby('id')['question_questions'].value_counts())

id                        question_questions                                              
0wcaegyyobblhhvfzwyibn0a  Please indicate your company's main area of business                17
                          Which of the following best describes your job function?            10
                          Reason for Attending the Event                                       5
                          What role do you play in the purchasing decision making process?     4
1o5g2tlsho4gxzk3usr06z1c  Reason for Attending the Event                                       5
                                                                                              ..
yzc27c0f8w2ortjytg54wtmh  What role do you play in the purchasing decision making process?     4
zcf116neh1ny42asu75xy0h2  Please indicate your company's main area of business                17
                          Which of the following best describes your job function?            10
                          Reason for